In [50]:
# external dependencies
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
import plotly.express as px

In [140]:
# constants file
from constants import *
from BuildGraphs import porcodio
porcodio()

ImportError: cannot import name 'porcodio' from 'BuildGraphs' (d:\Documents\repos\ARD_Project\BuildGraphs.py)

### Population Dataset

In [52]:
# load dataset for US population
df_pop = pd.read_csv('US_Population.csv')
df_pop.dropna()
df_pop.head()

,Year,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,2016,4860545,741522,6941072,2989918,39250017,5540545,3576452,952065,20612439,...,865454,6651194,27862596,3051217,624594,8411808,7288000,1831102,5778708,585501
1,2017,4874747,739786,7044008,3001345,39536653,5607154,3573880,961939,20928863,...,869666,6715984,28304596,3101833,623657,8470020,7423362,1818157,5790186,584910
2,2018,4887681,735139,7158024,3009733,39776830,5691287,3573297,971180,21244317,...,878698,6771631,28704330,3153550,624344,8517685,7535591,1804291,5807406,578668
3,2019,4903185,731158,7278717,3017804,39576757,5758736,3571520,981822,21538187,...,882235,6829174,29145505,3205958,624358,8565256,7614893,1792147,5822434,577601
4,2020,4921532,727890,7421401,3029887,39368078,5845526,3565287,990837,21899341,...,886667,6886834,29618533,3251617,623989,8616207,7693612,1778070,5837466,567025


### Accident Dataset

In [53]:

# import dataset for US accident
df_acc = pd.read_csv('US_Accidents.csv')

# remove unecessary columns
df_acc.drop(['Source', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Description', 'Airport_Code'], axis=1, inplace=True)

# remove null values
df_acc.dropna()

# sample a subset of the database (for efficency)
df_acc = df_acc.sample(n = 70000, random_state = 351)

# Convert 'Start_Time' and 'End_Time to datetime
df_acc['Start_Time'] = pd.to_datetime(df_acc['Start_Time'],format='ISO8601')
df_acc['End_Time'] = pd.to_datetime(df_acc['End_Time'],format='ISO8601')

# remove year 2023 because it is incomplete
df_acc = df_acc[(df_acc['Start_Time'].dt.year != 2023)]

df_acc.head()

,ID,Severity,Start_Time,End_Time,Distance(mi),Street,City,County,State,Zipcode,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
2909380,A-2919259,3,2018-02-28 19:10:31,2018-02-28 19:55:17,0.00,Carson St,Lakewood,Los Angeles,CA,90715,...,False,False,False,False,False,False,Night,Night,Night,Day
4123847,A-4154878,2,2022-11-22 16:09:03,2022-11-22 17:26:55,0.99,I-40 W,Nashville,Davidson,TN,37205,...,False,False,False,False,False,False,Day,Day,Day,Day
7006131,A-7055452,2,2020-06-24 20:28:00,2020-06-24 21:02:05,0.00,I-94 E,Minneapolis,Hennepin,MN,55412,...,False,False,False,False,False,False,Day,Day,Day,Day
1191012,A-1200791,2,2021-01-08 18:01:03,2021-01-08 18:47:15,0.00,S Choctaw Dr,Baton Rouge,East Baton Rouge,LA,70815-8901,...,False,False,False,False,True,False,Night,Night,Day,Day
453540,A-453553,2,2017-05-11 08:05:29,2017-05-11 08:34:56,0.02,Meridian Ave N,Seattle,King,WA,98133-8540,...,False,False,False,False,True,False,Day,Day,Day,Day


In [59]:
def SingleBarChart(df, TimeInterval):

    if (TimeInterval == 'Yearly'):
        yearly_counts = df['Start_Time'].dt.year.value_counts().sort_index()
        fig = px.bar(
            x = yearly_counts.index, 
            y = yearly_counts.values,          
            labels = {'x': 'Year', 'y': 'Number of Accidents'},
            title = 'Number of Accidents per Year'
        )
        return fig
    
    if (TimeInterval == 'Monthly'):
        monthly_counts = df['Start_Time'].dt.month.value_counts().sort_index()
        fig = px.bar(
            x = monthly_counts.index, 
            y = monthly_counts.values,          
            labels = {'x': 'Month', 'y': 'Number of Accidents'},
            title = 'Number of Accidents per Month'
        )
        return fig
   
    if (TimeInterval == 'Daily'):
        daily_counts = df['Start_Time'].dt.dayofweek.value_counts().sort_index()
        day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        fig = px.bar(
            x = [day_names[day] for day in daily_counts.index], 
            y = daily_counts.values,          
            labels = {'x': 'Day of the Week', 'y': 'Number of Accidents'},
            title = 'Number of Accidents per Day'
        )
        return fig
    
    if (TimeInterval == 'Hourly'):
        hourly_counts = df['Start_Time'].dt.hour.value_counts().sort_index()
        fig = px.bar(
            x = hourly_counts.index, 
            y = hourly_counts.values,          
            labels = {'x': 'Hour', 'y': 'Number of Accidents'},
            title = 'Number of Accidents per Hour'
        )
        return fig
    
def MultiBarChart(df, TimeInterval):
    #devide dataset on severity of the accident
    Severity1 = df[df.Severity == 1]
    Severity2 = df[df.Severity == 2]
    Severity3 = df[df.Severity == 3]
    Severity4 = df[df.Severity == 4]
    
    if (TimeInterval == 'Yearly'):
        categories = df['Start_Time'].dt.year.value_counts().sort_index().index

        values1 = Severity1['Start_Time'].dt.year.value_counts().sort_index()
        values2 = Severity2['Start_Time'].dt.year.value_counts().sort_index()
        values3 = Severity3['Start_Time'].dt.year.value_counts().sort_index()
        values4 = Severity4['Start_Time'].dt.year.value_counts().sort_index()


        trace1 = go.Bar(
            x=categories,
            y=values1.values,
            name='Series 1'
        )
        trace2 = go.Bar(
            x=categories,
            y=values2.values,
            name='Series 2'
        )
        trace3 = go.Bar(
            x=categories,
            y=values3.values,
            name='Series 3'
        )
        trace4 = go.Bar(
            x=categories,
            y=values4.values,
            name='Series 4'
        )

        # Create the figure
        fig = go.Figure(data=[trace1, trace2, trace3, trace4])

        # Update layout for better visualization
        fig.update_layout(
            title='Multiple Bar Chart',
            #xaxis=dict(title='Categories'),
            #yaxis=dict(title='Values'),
            barmode='group',  # This will group the bars side by side
            bargap=0.15,      # Gap between bars of adjacent location coordinates
            bargroupgap=0.1   # Gap between bars of the same location coordinate
        )

        return fig
    
    if (TimeInterval == 'Monthly'):
        categories = df['Start_Time'].dt.month.value_counts().sort_index().index

        values1 = Severity1['Start_Time'].dt.month.value_counts().sort_index()
        values2 = Severity2['Start_Time'].dt.month.value_counts().sort_index()
        values3 = Severity3['Start_Time'].dt.month.value_counts().sort_index()
        values4 = Severity4['Start_Time'].dt.year.value_counts().sort_index()

        trace1 = go.Bar(
            x=categories,
            y=values1.values,
            name='Series 1',
            
        )
        trace2 = go.Bar(
            x=categories,
            y=values2.values,
            name='Series 2',
            
        )
        trace3 = go.Bar(
            x=categories,
            y=values3.values,
            name='Series 3',
            
        )
        trace4 = go.Bar(
            x=categories,
            y=values4.values,
            name='Series 4',
            
        )

        # Create the figure
        fig = go.Figure(data=[trace1, trace2, trace3, trace4])
        # Update layout for better visualization
        fig.update_layout(
            title='Multiple Bar Chart',
            #xaxis=dict(title='Categories'),
            #yaxis=dict(title='Values'),
            barmode='group',  # This will group the bars side by side
            bargap=0.15,      # Gap between bars of adjacent location coordinates
            bargroupgap=0.1   # Gap between bars of the same location coordinate
        )

        return fig
    
    if (TimeInterval == 'Daily'):
        categories = df['Start_Time'].dt.dayofweek.value_counts().sort_index().index

        values1 = Severity1['Start_Time'].dt.dayofweek.value_counts().sort_index()
        values2 = Severity2['Start_Time'].dt.dayofweek.value_counts().sort_index()
        values3 = Severity3['Start_Time'].dt.dayofweek.value_counts().sort_index()
        values4 = Severity4['Start_Time'].dt.year.value_counts().sort_index()

        trace1 = go.Bar(
            x=categories,
            y=values1.values,
            name='Series 1'
        )
        trace2 = go.Bar(
            x=categories,
            y=values2.values,
            name='Series 2'
        )
        trace3 = go.Bar(
            x=categories,
            y=values3.values,
            name='Series 3'
        )

        trace4 = go.Bar(
            x=categories,
            y=values4.values,
            name='Series 4'
        )

        # Create the figure
        fig = go.Figure(data=[trace1, trace2, trace3, trace4])
        # Update layout for better visualization
        fig.update_layout(
            title='Multiple Bar Chart',
            #xaxis=dict(title='Categories'),
            #yaxis=dict(title='Values'),
            barmode='group',  # This will group the bars side by side
            bargap=0.15,      # Gap between bars of adjacent location coordinates
            bargroupgap=0.1   # Gap between bars of the same location coordinate
        )

        return fig
    
    if (TimeInterval == 'Hourly'):
        categories = df['Start_Time'].dt.hour.value_counts().sort_index().index

        values1 = Severity1['Start_Time'].dt.hour.value_counts().sort_index()
        values2 = Severity2['Start_Time'].dt.hour.value_counts().sort_index()
        values3 = Severity3['Start_Time'].dt.hour.value_counts().sort_index()
        values4 = Severity4['Start_Time'].dt.year.value_counts().sort_index()

        trace1 = go.Bar(
            x=categories,
            y=values1.values,
            name='Series 1'
        )
        trace2 = go.Bar(
            x=categories,
            y=values2.values,
            name='Series 2'
        )
        trace3 = go.Bar(
            x=categories,
            y=values3.values,
            name='Series 3'
        )
        trace4 = go.Bar(
            x=categories,
            y=values4.values,
            name='Series 4'
        )

        # Create the figure
        fig = go.Figure(data=[trace1, trace2, trace3, trace4])
        # Update layout for better visualization
        fig.update_layout(
            title='Multiple Bar Chart',
            #xaxis=dict(title='Categories'),
            #yaxis=dict(title='Values'),
            barmode='group',  # This will group the bars side by side
            bargap=0.15,      # Gap between bars of adjacent location coordinates
            bargroupgap=0.1   # Gap between bars of the same location coordinate
        )

        return fig

In [115]:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


__Dashboard Layout__

In [117]:
app.layout = dbc.Container([

    # title
    dbc.Row(dbc.Col(html.H1("USA Accidents"), width="auto"), justify="center",), 
    html.Br(),

    # top buttons 
    dbc.Row([

        # time interval select
        dbc.Col([
            dbc.Select(
                options=[
                    {"label":"Monthly", "value":"Monthly"}, 
                    {"label":"Yearly", "value":"Yearly"}, 
                    {"label":"Dayly", "value":"Dayly"},
                    {"label":"Hourly", "value":"Hourly"}  
                ],
                value='Monthly',
                id="Time-Interval-Select"
            )
        ]),

        # view mode select
        dbc.Col([
            dbc.RadioItems(
                options=[
                    {"label":"Grouped", "value":"Grouped"}, 
                    {"label":"Separated", "value":"Separated"}
                ],
                value = 'Grouped',
                id='ViewMode-Select'
            )
        ])
    ]),

    # first graph row
    dbc.Row([

        # top-left graph
        dbc.Col([
            # barchart
            dcc.Graph(id='Temporal-Distribution-acc-graph')
        ])

    ])

])

__Top-left graph callback__

In [118]:
@app.callback(Output("Temporal-Distribution-acc-graph", "figure"),[Input("Time-Interval-Select", "value"), Input("ViewMode-Select", "value")])
def TDABarchart(Time_Interval, ViewMode):
    if(ViewMode == 'Grouped'):
        fig = bgf.SingleBarChart(df_acc, Time_Interval)
        return fig
    else:
        if(ViewMode == 'Grouped'):
            fig = MultiBarChart(df_acc, Time_Interval)
            return fig
        else:
            return 0

In [119]:
app.run(debug=True)

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[118], line 4, in TDABarchart(
    Time_Interval='Monthly',
    ViewMode='Grouped'
)
      1 @app.callback(Output("Temporal-Distribution-acc-graph", "figure"),[Input("Time-Interval-Select", "value"), Input("ViewMode-Select", "value")])
      2 def TDABarchart(Time_Interval, ViewMode):
      3     if(ViewMode == 'Grouped'):
----> 4         fig = bgf.SingleBarChart(df_acc, Time_Interval)
        df_acc =                 ID  Severity          Start_Time            End_Time  \
2909380  A-2919259         3 2018-02-28 19:10:31 2018-02-28 19:55:17   
4123847  A-4154878         2 2022-11-22 16:09:03 2022-11-22 17:26:55   
7006131  A-7055452         2 2020-06-24 20:28:00 2020-06-24 21:02:05   
1191012  A-1200791         2 2021-01-08 18:01:03 2021-01-08 18:47:15   
453540    A-453553         2 2017-05-11 08:05:29 2017-05-11 08:34:56   
...

http://127.0.0.1:8050